In [16]:
import numpy as np
import pandas as pd

### PARTIE 1 : Exploration de la base de données

In [17]:
df = pd.read_csv('csv_files/regularite-mensuelle-tgv-aqst.csv',sep=';')
print('Information : ', '\n', df.info())
print('Description statistique de chaque colonne : ', '\n', df.describe())
print('Donées manquantes : ', '\n', df.isnull().sum())
print('Type des données : ', '\n', df.dtypes)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8154 entries, 0 to 8153
Data columns (total 26 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   date                                  8154 non-null   object 
 1   service                               8154 non-null   object 
 2   gare_depart                           8154 non-null   object 
 3   gare_arrivee                          8154 non-null   object 
 4   duree_moyenne                         8154 non-null   int64  
 5   nb_train_prevu                        8154 non-null   int64  
 6   nb_annulation                         8154 non-null   int64  
 7   commentaire_annulation                0 non-null      float64
 8   nb_train_depart_retard                8154 non-null   int64  
 9   retard_moyen_depart                   8154 non-null   float64
 10  retard_moyen_tous_trains_depart       8154 non-null   float64
 11  commentaire_retar

### PARTIE 2 : Traitement de la base de données
#### Gestion des commentaires

On observe que les seules valeurs manquante sont pour les features commentaires : commentaire_annulation, commentaire_retards_depart
et commentaire_retard_arrivee
Pour une première approche simplificatrice du dataset, on prend la décision de supprimier le

In [18]:
df_clean = (
    df.copy()
        .drop(
            ["commentaire_annulation", "commentaire_retards_depart", "commentaires_retard_arrivee"],
            axis=1,
        )
)

df_clean.columns

Index(['date', 'service', 'gare_depart', 'gare_arrivee', 'duree_moyenne',
       'nb_train_prevu', 'nb_annulation', 'nb_train_depart_retard',
       'retard_moyen_depart', 'retard_moyen_tous_trains_depart',
       'nb_train_retard_arrivee', 'retard_moyen_arrivee',
       'retard_moyen_tous_trains_arrivee', 'nb_train_retard_sup_15',
       'retard_moyen_trains_retard_sup15', 'nb_train_retard_sup_30',
       'nb_train_retard_sup_60', 'prct_cause_externe', 'prct_cause_infra',
       'prct_cause_gestion_trafic', 'prct_cause_materiel_roulant',
       'prct_cause_gestion_gare', 'prct_cause_prise_en_charge_voyageurs'],
      dtype='object')

#### Gestion de la colonne service

In [19]:
df_clean["service"].unique()

array(['National', 'International'], dtype=object)

On observe que la colonne ne possède que deux valeurs categorielles, nous allons faire un "one-hot" encoding et creer une feature booléenne "national_service"

In [20]:
df_clean["national_service"] = np.where(df_clean["service"]=="National", 1,0)
df_clean.drop("service", axis=1, inplace=True)

,date,gare_depart,gare_arrivee,duree_moyenne,nb_train_prevu,nb_annulation,nb_train_depart_retard,retard_moyen_depart,retard_moyen_tous_trains_depart,nb_train_retard_arrivee,...,retard_moyen_trains_retard_sup15,nb_train_retard_sup_30,nb_train_retard_sup_60,prct_cause_externe,prct_cause_infra,prct_cause_gestion_trafic,prct_cause_materiel_roulant,prct_cause_gestion_gare,prct_cause_prise_en_charge_voyageurs,national_service
0,2018-01,BORDEAUX ST JEAN,PARIS MONTPARNASSE,141,870,5,289,11.247809,3.693179,147,...,6.511118,44,8,36.134454,31.092437,10.924370,15.966387,5.042017,0.840336,1
1,2018-01,LA ROCHELLE VILLE,PARIS MONTPARNASSE,165,222,0,8,2.875000,0.095796,34,...,5.696096,5,0,15.384615,30.769231,38.461538,11.538462,3.846154,0.000000,1
2,2018-01,PARIS MONTPARNASSE,QUIMPER,220,248,1,37,9.501351,1.003981,26,...,7.548387,17,7,26.923077,38.461538,15.384615,19.230769,0.000000,0.000000,1
3,2018-01,PARIS MONTPARNASSE,ST MALO,156,102,0,12,19.912500,1.966667,13,...,6.724757,6,4,23.076923,46.153846,7.692308,15.384615,7.692308,0.000000,1
4,2018-01,PARIS MONTPARNASSE,ST PIERRE DES CORPS,61,391,2,61,7.796995,0.886889,71,...,3.346487,6,0,21.212121,42.424242,9.090909,21.212121,6.060606,0.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8149,2023-06,STRASBOURG,PARIS EST,114,492,2,273,11.989927,6.742041,105,...,35.446667,31,10,7.619048,5.714286,55.238095,18.095238,7.619048,5.714286,1
8150,2023-06,TOULOUSE MATABIAU,PARIS MONTPARNASSE,273,215,0,24,28.730556,3.181240,36,...,76.688889,24,10,13.888889,33.333333,8.333333,19.444444,8.333333,16.666667,1
8151,2023-06,TOURS,PARIS MONTPARNASSE,78,192,1,20,31.281667,3.276353,32,...,97.394872,7,5,14.285714,21.428571,28.571429,21.428571,3.571429,10.714286,1
8152,2023-06,VALENCE ALIXAN TGV,PARIS LYON,133,440,2,347,13.517051,10.271499,116,...,47.424479,62,22,26.724138,19.827586,27.586207,7.758621,7.758621,10.344828,1


#### Gestion de la date

Nous allons encoder la date en pd.Datetime64, ce format nous permettra de realiser des analyses en timeseries.

In [25]:
df_clean["date"] = pd.to_datetime(df_clean["date"])

#### Gestion des villes

Nous allons tout d'abord créer la feature trajet, réunissant les gares de départ et d'arrivee

In [27]:
df_clean["trajet"] = df_clean["gare_depart"] + " - " + df_clean["gare_arrivee"]

Nous voulons cependant conserver les deux features d'origine. En effet la gare du départ ou d'arrivée peut avoir un impact sur le retard indépendemment du trajet.

Ici, il serait judicieux d'appliquer un "one-hot encoding" sur les deux gares ainsi que notre nouvelle variable, cependant dans un objectif de lisibilité du dataframe, nous allons conserver cette étape pour la fin du pre-processing. En effet cet encodage créera des dizaines de colonnes supplémentaires.